In [ ]:
#READ ME
##################################################################################################################################################
#Determine the number of passes until the satellite repeats its ground trace
#Definiton of pass break should be definied in the satellite's properties; whether it is ascending or descedning at the equator
#This script loads the Passes data provider. This data provider gives the pass number, latitude at pass break and longitude at pass break, 
#as well as the start and stop times of each pass interval. The latitude at the pass break is ~zero, since the pass break is defined at the equator. 
#Whether the pass break is defined as ascending or descending only depends on the users prefered convention, as the data provider provides passes only at
#the node where the break is defined. This means that the latitude and longitude rates do not have to be checked for matching direction of pass. 
#The logic implemented checks for a repeating logitude at pass break value. 


#User inputs:

#The user has to input the name of the satellite, as well as the reference pass number to begin the search from. A reference data frame is displayed 
#so that the user can pick a pass number without having to generate a custom report inside of STK. 

In [1]:
import datetime as dt
import numpy as np
import os
import pandas as pd

# Get reference to running STK instance
from agi.stk12.stkdesktop import STKDesktop

stk = STKDesktop.AttachToApplication()

# Get the IAgStkObjectRoot interface
root = stk.Root

In [2]:
#Get a reference to the scenario object
scenario = root.CurrentScenario

##################################################################################################################################
#Get Satellite Object in scenario#
#Change the name of the satellite in ''   #

Satellite=scenario.Children.GetItemByName('RCM-1_44322')

#Get the Passes data provider
passD=Satellite.DataProviders.Item("Passes")
passDgroup=passD.ExecElements(scenario.StartTime, scenario.StopTime,['Pass Number', 'Start Time', 'End Time','Lat of Pass Break','Lon of Pass Break'])
#turn into a pandas data frame
passDataSet = passDgroup.DataSets.ToPandasDataFrame()
passDataSet.drop([0],inplace = True)
#display data frame for reference
passDataSet

,pass number,start time,end time,lat of pass break,lon of pass break
1,28163,12 Aug 2024 17:26:14.494977581,12 Aug 2024 19:02:46.514954408,-0.0873947735225083,8.601422286373392
2,28164,12 Aug 2024 19:02:46.514954408,12 Aug 2024 20:39:18.533603963,-0.08727234622554693,-15.532017288934808
3,28165,12 Aug 2024 20:39:18.533603963,12 Aug 2024 22:15:50.550926253,-0.08714981951087282,-39.66545134781159
4,28166,12 Aug 2024 22:15:50.550926253,12 Aug 2024 23:52:22.566921262,-0.08702719257195864,-63.798879881983495
5,28167,12 Aug 2024 23:52:22.566921262,13 Aug 2024 01:28:54.581588974,-0.08690446607985204,-87.93230281204092
...,...,...,...,...,...
354,28516,5 Sep 2024 09:21:15.076499704,5 Sep 2024 10:57:46.627912830,-0.03837724529765448,129.84227401603937
355,28517,5 Sep 2024 10:57:46.627912830,5 Sep 2024 12:34:18.177999064,-0.0382242965810016,105.71079509437088
356,28518,5 Sep 2024 12:34:18.177999064,5 Sep 2024 14:10:49.726758400,-0.03807127575269202,81.57932169859559
357,28519,5 Sep 2024 14:10:49.726758400,5 Sep 2024 15:47:21.274190835,-0.037918183457602166,57.44785387055359


In [3]:


#####################################################################################################################################################
#Reference input you imput to start to search
Input_PassNumber='28163'
#tolerance for the longitude at pass break match
sigma=0.8

#The row of the dataset correcsponding to the user input time
reference_row = passDataSet.loc[passDataSet['pass number'] == Input_PassNumber]
passNumber_ref = float(reference_row['pass number'])

#The index of the row corresponding to the matched row
index_input = reference_row.index


#function to get the time of repeat ground trace
def get_repeated_pass_number():
    #locate where the time is the reference time
    reference_row = passDataSet.loc[passDataSet['pass number'] == Input_PassNumber]
    
    #turn the value into a number
    #pass number and lon at pass break corresponding to the reference row
    lon_ref = float(reference_row['lon of pass break'])
    passNumber_ref = float(reference_row['pass number'])

    #threshold bounds for lon search
    upperThreshLon = lon_ref + sigma
    lowerThreshLon = lon_ref - sigma

    #loop through the data frame rows
    for index, passD in passDataSet.iterrows():

        #only loop through the rows after the input time, ignore everything before
        if index > index_input[0] :
            #Declare data for current step
            Time=passD.at['start time']
            Lon=float(passD.at['lon of pass break'])
            Pass=passD.at['pass number']
            
            repeatLon=lon_ref
            
            #apply logic
            if Lon <= upperThreshLon and Lon >= lowerThreshLon :
                TimeOfRepeat = Time
                RepeatPassNumber = Pass
                return RepeatPassNumber, TimeOfRepeat

In [4]:
#run function and assign the pass number and time of the repeat 
repeat_pass, repeat_time = get_repeated_pass_number()
repeat_pass=float(repeat_pass)
#print for reference
print(repeat_pass)
print(repeat_time)

28342.0
24 Aug 2024 17:25:24.923887527


In [5]:
#subtracting the pass number at the reference time from the pass number at the repeat time to get the number of passes with
#repect to the reference time
Number_of_passes = repeat_pass - passNumber_ref
Number_of_passes

179.0